In [285]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [286]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [287]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    # 检查边数是否合理
    if num_edges < num_nodes or num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges must be at least num_nodes and no more than num_nodes * (num_nodes - 1) // 2.")

    # 生成节点列表
    nodes = ['r'] + [chr(i) for i in range(97, 97 + num_nodes - 1)]

    # 创建一个环来确保每个节点都在环上
    cycle_edges = [(nodes[i], nodes[(i + 1) % num_nodes]) for i in range(num_nodes)]
    
    # 生成除环路外其他节点间可能的边
    possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j and (nodes[j], nodes[i]) not in cycle_edges]

    # 从可能的边中随机选择剩余的边
    np.random.shuffle(possible_edges)
    remaining_edges_count = num_edges - len(cycle_edges)
    selected_edges = cycle_edges + possible_edges[:remaining_edges_count]
    
    # 为每条边生成随机成本
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# 使用该函数生成一个随机有向图
num_nodes = 6  # 节点数量，包括 'r'
num_edges = 10  # 总边数，至少等于节点数
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['r', 'a', 'b', 'c', 'd', 'e']
Directed edges with costs: {('r', 'a'): 3, ('a', 'b'): 2, ('b', 'c'): 1, ('c', 'd'): 6, ('d', 'e'): 3, ('e', 'r'): 5, ('e', 'b'): 7, ('a', 'd'): 6, ('d', 'a'): 1}


In [288]:

#nodes = ['a','b','c','d','r']
#edges_with_costs ={
    #('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    #('b','r'):4,('d','b'):6,('r','c'):2,
    #('c','d'):7,('d','r'):5
#}
'''
nodes = ['a','b','c','d','r']
edges_with_costs = {('b','d'):5,('a','b'): 8, ('b','c'): 9,
                    ('a','c'): 8, ('b','r'): 9, ('c','r'): 8,
                    ('a','r'): 5, ('a','d'): 7, ('c','d'): 3}
'''

"\nnodes = ['a','b','c','d','r']\nedges_with_costs = {('b','d'):5,('a','b'): 8, ('b','c'): 9,\n                    ('a','c'): 8, ('b','r'): 9, ('c','r'): 8,\n                    ('a','r'): 5, ('a','d'): 7, ('c','d'): 3}\n"

In [289]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
x

{'r': {'r': x_r_r, 'a': x_r_a, 'b': x_r_b, 'c': x_r_c, 'd': x_r_d, 'e': x_r_e},
 'a': {'r': x_a_r, 'a': x_a_a, 'b': x_a_b, 'c': x_a_c, 'd': x_a_d, 'e': x_a_e},
 'b': {'r': x_b_r, 'a': x_b_a, 'b': x_b_b, 'c': x_b_c, 'd': x_b_d, 'e': x_b_e},
 'c': {'r': x_c_r, 'a': x_c_a, 'b': x_c_b, 'c': x_c_c, 'd': x_c_d, 'e': x_c_e},
 'd': {'r': x_d_r, 'a': x_d_a, 'b': x_d_b, 'c': x_d_c, 'd': x_d_d, 'e': x_d_e},
 'e': {'r': x_e_r, 'a': x_e_a, 'b': x_e_b, 'c': x_e_c, 'd': x_e_d, 'e': x_e_e}}

In [290]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob

Minimize_Cost:
MINIMIZE
2*x_a_b + 6*x_a_d + 1*x_b_c + 6*x_c_d + 1*x_d_a + 3*x_d_e + 7*x_e_b + 5*x_e_r + 3*x_r_a + 0
VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_a <= 1 Integer
0 <= x_d_e <= 1 Integer
0 <= x_e_b <= 1 Integer
0 <= x_e_r <= 1 Integer
0 <= x_r_a <= 1 Integer

In [291]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
        #prob +=pulp.lpSum(x[u,v] for u in nodes if (v,u) in edges_with_costs) == 1
prob 

Minimize_Cost:
MINIMIZE
2*x_a_b + 6*x_a_d + 1*x_b_c + 6*x_c_d + 1*x_d_a + 3*x_d_e + 7*x_e_b + 5*x_e_r + 3*x_r_a + 0
SUBJECT TO
_C1: x_d_a + x_r_a = 1

_C2: x_a_b + x_e_b = 1

_C3: x_b_c = 1

_C4: x_a_d + x_c_d = 1

_C5: x_d_e = 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_a <= 1 Integer
0 <= x_d_e <= 1 Integer
0 <= x_e_b <= 1 Integer
0 <= x_e_r <= 1 Integer
0 <= x_r_a <= 1 Integer

In [292]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1
prob

Minimize_Cost:
MINIMIZE
2*x_a_b + 6*x_a_d + 1*x_b_c + 6*x_c_d + 1*x_d_a + 3*x_d_e + 7*x_e_b + 5*x_e_r + 3*x_r_a + 0
SUBJECT TO
_C1: x_d_a + x_r_a = 1

_C2: x_a_b + x_e_b = 1

_C3: x_b_c = 1

_C4: x_a_d + x_c_d = 1

_C5: x_d_e = 1

_C6: x_e_r = 0

_C7: x_r_a >= 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_a_d <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_a <= 1 Integer
0 <= x_d_e <= 1 Integer
0 <= x_e_b <= 1 Integer
0 <= x_e_r <= 1 Integer
0 <= x_r_a <= 1 Integer

In [293]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)
subset

[('r',),
 ('r', 'a'),
 ('r', 'b'),
 ('r', 'c'),
 ('r', 'd'),
 ('r', 'e'),
 ('r', 'a', 'b'),
 ('r', 'a', 'c'),
 ('r', 'a', 'd'),
 ('r', 'a', 'e'),
 ('r', 'b', 'c'),
 ('r', 'b', 'd'),
 ('r', 'b', 'e'),
 ('r', 'c', 'd'),
 ('r', 'c', 'e'),
 ('r', 'd', 'e'),
 ('r', 'a', 'b', 'c'),
 ('r', 'a', 'b', 'd'),
 ('r', 'a', 'b', 'e'),
 ('r', 'a', 'c', 'd'),
 ('r', 'a', 'c', 'e'),
 ('r', 'a', 'd', 'e'),
 ('r', 'b', 'c', 'd'),
 ('r', 'b', 'c', 'e'),
 ('r', 'b', 'd', 'e'),
 ('r', 'c', 'd', 'e'),
 ('r', 'a', 'b', 'c', 'd'),
 ('r', 'a', 'b', 'c', 'e'),
 ('r', 'a', 'b', 'd', 'e'),
 ('r', 'a', 'c', 'd', 'e'),
 ('r', 'b', 'c', 'd', 'e')]

In [294]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1  

In [295]:
prob.solve()
print ("Status:", LpStatus[prob.status])

Status: Optimal


In [296]:
print ("Optimal Solution")
for i in nodes:
	for j in nodes:
		if x[i][j].value() == 1:
			print ("(%s,%s)"%(i,j))  


Optimal Solution
(r,a)
(a,b)
(b,c)
(c,d)
(d,e)
